In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
from scipy.signal import *
import matplotlib.pyplot as plt
#from AdaBandFlt import *

%matplotlib tk

In [2]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'


# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')
df

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,El 41,El 42,El 44,El 54,El 55,El 56,El 58,El 86,El 15,El 51
%t,,,,,,,,,,
0.00,0.92,-1.46,2.32,-0.61,0.06,-1.04,8.06,-0.24,0.73,-0.55
0.02,0.49,-1.53,1.59,-0.31,0.73,-1.22,6.29,-0.85,0.92,-1.40
0.04,0.43,-1.34,0.92,-0.37,1.16,-1.10,4.15,-0.92,0.85,-2.08
0.06,0.85,-0.85,0.37,-0.85,1.04,-0.92,2.08,-0.73,0.61,-2.44
0.08,1.46,-0.37,-0.12,-1.46,0.31,-0.73,0.55,-0.55,0.18,-2.38
...,...,...,...,...,...,...,...,...,...,...
299999.90,1.34,0.73,4.21,-1.95,-4.76,-1.28,-6.96,-3.60,-0.31,2.50
299999.92,1.59,0.49,4.09,-2.44,-3.42,-1.34,-6.04,-4.09,0.00,2.93
299999.94,2.08,0.37,3.66,-3.48,-1.95,-1.53,-5.13,-4.64,0.18,2.93


In [3]:
def test_valid_window(window, test_level = 5):
    """
    window : the window in the signal that has to be tested
    
    This funtion test the window to insure that it doesn't contain the signal of interest (spike)
    """
    #non zero ?
    second = np.percentile(window, 2)
    thirtyth = np.percentile(window, 30)
    #print(str(second) + "\t" + str(thirtyth) + "\t" + str(second/thirtyth))
    if abs(second/thirtyth) < test_level : 
        return True
    else : 
        return False

In [9]:
def init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25):
    
    if estimator_type == "RMS":
        return init_noise_levels_RMS(signal, fs, 
                      noise_window_size = noise_window_size,
                      required_valid_windows = required_valid_windows,
                      old_noise_level_propagation = old_noise_level_propagation, 
                      test_level = test_level,
                      percentile_value = percentile_value)
        
    elif estimator_type == "MAD":
        return init_noise_levels_MAD(signal, fs, 
                      noise_window_size = noise_window_size,
                      required_valid_windows = required_valid_windows,
                      old_noise_level_propagation = old_noise_level_propagation, 
                      test_level = test_level,
                      percentile_value = percentile_value)
    
    else: return None
    
    
def init_noise_levels_RMS(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      percentile_value = 25):
    
    nb_valid_windows = 0
    list_RMS = []
    noise_levels = []
    
    noise_level = -1
    
     
    #boucle en indice
#    for window_index in range(0,len(signal)-(len(signal)%int(fs*noise_window_size)),int(fs*noise_window_size)):
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_RMS, percentile_value)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                else :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                NX = np.percentile(list_RMS, percentile_value)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*NX
                noise_level = new_noise_level
            noise_levels.append(noise_level)
            
    #cas ou il n'y a pas eut 100 fenetres de bruit valides rencontrees
    if noise_level == -1:
        
        #cas ou aucune fenetre valide n'a ete rencontree
        if noise_levels == []:
            for elm in range(0, len(signal), int(fs*noise_window_size)):
                noise_levels.append(0)
            
        else:
            noise_level = np.percentile(list_RMS, percentile_value)
            for elm in range(0, len(signal), int(fs*noise_window_size)):
                noise_levels.append(noise_level)
        
    
    noise_levels.append(noise_level)        
    plt.figure()
    plt.plot(list_RMS)
    plt.xlabel('Time Windows')
    plt.title('RMS values')
    plt.grid(True)
                
    return noise_levels

def init_noise_levels_MAD(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      percentile_value = 25):
    
    nb_valid_windows = 0
    list_MAD = []
    noise_levels = []
     
    #boucle en indice
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                ###RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index: window_index + int(fs*noise_window_size)])
                list_MAD.append(MAD)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_MAD, percentile_value)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    ###RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                    MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index:])
                else :
                    ###RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                    MAD = scipy.stats.median_absolute_deviation(signal.iloc[window_index: window_index + int(fs*noise_window_size)])
                list_MAD.append(MAD)
                NX = np.percentile(list_MAD, percentile_value)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*NX
                noise_level = new_noise_level
            noise_levels.append(noise_level)
    
    noise_levels.append(noise_levels)        
    plt.figure()
    plt.plot(list_MAD)
    plt.xlabel('Time Windows')
    plt.title('MAD values')
    plt.grid(True)
                
    return noise_levels

In [10]:
#find spike

def find_spike(signal, initial_index, noise_levels, fs, spike_info, 
               window_size = 0.002, 
               noise_window_size = 0.01,
               threshold_factor = 3.5,
               maxseparation = 0.0008):
    
    offset_index = int(np.round(signal.index[0]*fs/1000))
    
    if initial_index < len(signal) + offset_index:
        i = initial_index
        for value in signal.iloc[initial_index-offset_index:]:
            threshold = threshold_factor*noise_levels[int(np.round((i/fs)//noise_window_size))]
            if value > threshold:
                while(True):
                    if i > initial_index + window_size*fs:
                        b_point = int(np.round(i - window_size*fs)) - offset_index
                    else :
                        b_point = initial_index - offset_index
                    if i < len(signal)+offset_index - window_size*fs-1:
                        e_point = int(np.round(i + window_size*fs)) - offset_index
                    else :
                        e_point = len(signal)

                    highest_value = signal.iloc[b_point: e_point].max()
                    
                    if highest_value == value : 
                        break
                    
                    else:
                        i = int(np.round(signal.iloc[b_point: e_point].idxmax()*fs/1000))
                        value = signal.iloc[i-offset_index] 
                
                
                
                if i == initial_index and i-offset_index-1>=0:
                    if signal.iloc[i-offset_index-1]>signal.iloc[i-offset_index]:
                        #print("Cas on est au mur gauche")
                        #décaler la limite de gauche jusqu'à la prochaine valeur en dessous du seuil
                        while signal.iloc[i-offset_index]>threshold:
                            i += 1
                        return i
               
                i_min = -33
                
                #partir à la recherche d'un min du spike
                for k in range(int(np.round(maxseparation*fs))):
                    if (i-offset_index + k) < len(signal)-1:
                        if signal.iloc[i-offset_index+k] < -threshold and signal.iloc[i-offset_index+k]<signal.iloc[i-offset_index+k+1]:
                            if checkminlocal(signal, "right",i+k,offset_index,3):
                            #if(signal.iloc[i-offset_index+k+1]<signal.iloc[i-offset_index+k+2]):
                                i_min = i+k
                                break
                    if (i-offset_index - k) > 0:
                        if signal.iloc[i-offset_index-k]<-threshold and signal.iloc[i-offset_index-k]<signal.iloc[i-offset_index-k-1]:
                            if checkminlocal(signal, "left",i-k,offset_index,3):
                            #if(signal.iloc[i-offset_index-k-1]<signal.iloc[i-offset_index-k-2]):
                                i_min = i-k
                                break
                
                
                if i_min == -33:
                    #on a pas rencontré de spike
                    while signal.iloc[i-offset_index]>threshold:
                        i += 1
                    return i
                
                else:
                    #récolte infos du spike
                    #on cherche ensuite le premier depassement positif (indpos) et négatif (indneg)
                    indpos = i
                    indneg = i_min
                    
                    while signal.iloc[indpos-offset_index] > threshold:
                        indpos -= 1
                        if indpos == offset_index-1:
                            break
                    indpos += 1 #on obtient l'indice du premier dépassement positif
                    while signal.iloc[indneg-offset_index] < -threshold:
                        indneg -= 1
                        if indneg == offset_index-1:
                            break
                    indneg += 1 #on obtient l'indice du premier dépassement négatif
                    
                    #on cherhce le zéro entre les deux "bosses"
                    indzero = i
                    if(i>i_min): #le max est après
                        while signal.iloc[indzero-offset_index] > 0:
                            indzero -= 1
                    else:
                        while signal.iloc[indzero-offset_index] > 0:
                            indzero += 1
                        indzero -= 1
                    
                         # indice max, indice min, indice 1er depasssement pos, indice 1er depassement nég, 
                            # premier dépassement, indice du zero central,distance entre max et min, 
                            #variation d'amplitude entre min et max
                            
                    spike_info.append([i, i_min, indpos, indneg, min(indpos,indneg), indzero, i-i_min, 
                                       (signal.iloc[i-offset_index]-signal.iloc[i_min-offset_index])])
                    return i+int(np.round(window_size*fs))
                
                break  
            i += 1

    return -44

def checkminlocal(local_signal, sens, supposed_i_min,offset_index, nb_index_research=3):
    if(sens == "right"):
        k = 0
        while k <= nb_index_research:
        #for k in range(nb_index_research):
            if((local_signal.iloc[supposed_i_min-offset_index + k]) > (local_signal.iloc[supposed_i_min-offset_index + k + 1])):
                return False
            k += 1
        return True
    elif(sens == "left"):
        k = 0
        while k <= nb_index_research:
        #for k in range(nb_index_research):
            if((local_signal.iloc[supposed_i_min-offset_index - k]) > (local_signal.iloc[supposed_i_min-offset_index - k - 1])):
                return False
            k+=1
        return True
    else:
        return False



#vérifier que le deuxième extrêmum n'est pas collé au mur de gauche ni de droite
            
def find_spikes(signal, noise_levels, fs, 
               window_size = 0.002, 
               noise_window_size = 0.01,
               threshold_factor = 3.5,
               maxseparation = 0.0008):
    
    initial = int(np.round(signal.index[0]*fs/1000))
    spike_info = []
    
    while initial != -44:
        initial = find_spike(signal, initial, noise_levels, fs, spike_info,
                             window_size = window_size, 
                             noise_window_size = noise_window_size,
                             threshold_factor = threshold_factor,
                             maxseparation = 0.0008)
        
    df_spike_info = pd.DataFrame(spike_info)
    
    df_spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']

    return df_spike_info

In [11]:
#record spike

def record_spikes(signal, fs, spike_info,
                  align_method,
                  t_before = 0.002,
                  t_after = 0.002):
    
    if (align_method in spike_info.columns) == False:
        print("align_method is incorrect, please choose one of the following :" + str(spike_info.columns))
        return None
    
    else:
        spike_centers = spike_info[align_method].values
        
    t_b = int(np.round(fs*(t_before)))
    t_a = int(np.round(fs*(t_after)))
    
    data = np.array([[float(x) for x in range(t_b+t_a+1)]])
    
    initial_index = int(np.round(signal.index[0]*fs/1000))
    
    for center in spike_centers:
        if center < t_b + initial_index:
            spike = [0 for i in range(0, t_b-(center-initial_index))]
            spike = np.concatenate((spike, signal.values[:center + t_a - initial_index]))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > len(signal)-t_a + initial_index:
            spike = signal.values[center - t_b - initial_index:]
            spike = np.concatenate((spike,[0 for i in range(0, t_a - (len(signal) + initial_index-center))]))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            spike = signal.values[center - t_b - initial_index: center + t_a + 1 - initial_index]
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [12]:
def record_spikes_oneline(signal, fs, spike_info,
                  align_method,
                  t_before = 0.001,
                  t_after = 0.002):

    if (align_method in spike_info.columns) == False:
        print("align_method is incorrect, please choose one of the following :" + str(spike_info.columns))
        return None
    
    else:
        spike_centers = spike_info[align_method].values
        
    offset_index = int(np.round(signal.index[0]*fs/1000))
    
    t_b = int(np.round(fs*(t_before)))
    t_a = int(np.round(fs*(t_after)))
    
    data = np.array(['NaN' for x in range(len(signal))])
    data = data.astype(float)
    times = np.array(['NaN' for x in range(len(signal))])
    times = times.astype(pd.Timestamp)
    
    for center in spike_centers:
        if center < t_b + offset_index:
            data[:center + t_a - offset_index] = signal.values[:center + t_a - offset_index]
            times[:center + t_a - offset_index] = signal.index[:center + t_a - offset_index]
            
        elif center > len(signal) - t_a + offset_index:
            data[center - t_b - offset_index:] = signal.values[center - t_b - offset_index:]
            times[center - t_b - offset_index:] = signal.index[center - t_b - offset_index:]
            
        else :
            data[center - t_b - offset_index: center + t_a + 1 - offset_index] = signal.values[center - t_b - offset_index: center + t_a + 1 - offset_index]
            times[center - t_b - offset_index: center + t_a + 1 - offset_index] = signal.index[center - t_b - offset_index: center + t_a + 1 - offset_index]

    spike_data_oneline = pd.DataFrame(data, index = times.astype(float))
    
    return spike_data_oneline

In [13]:
from random import randint

def print_spikes(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = 25000,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -50,
                 y_lim_max = 60):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.iloc[:,0]-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
    
"""
print_spikes(spike_data,
             t_before_alignement = 0.0015,
             first_spike = 1,
             last_spike = 20,
             fs = 25000,
             y_lim_min = -50,
             y_lim_max = 60)
             
print_spikes(spike_data,
             t_before_alignement = 0.0015,
             fs = 25000,
             randomize = True,
             nb_spike = 20,
             y_lim_min = -50,
             y_lim_max = 60)
"""

'\nprint_spikes(spike_data,\n             t_before_alignement = 0.0015,\n             first_spike = 1,\n             last_spike = 20,\n             fs = 25000,\n             y_lim_min = -50,\n             y_lim_max = 60)\n             \nprint_spikes(spike_data,\n             t_before_alignement = 0.0015,\n             fs = 25000,\n             randomize = True,\n             nb_spike = 20,\n             y_lim_min = -50,\n             y_lim_max = 60)\n'

In [15]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

#full_signal = df.iloc[:,1] #Entire recording from electrode 58
full_signal = df.iloc[:1000000,6] #Entire recording from electrode 58

electrode_ref = df.iloc[:1000000,8]

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 50000.0
lowcut = 100.0
highcut = 5000.0


#y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)
y = butter_bandpass_filter(df.iloc[:1000000,6], lowcut, highcut, fs, order=6)
y_ref = butter_bandpass_filter(df.iloc[:1000000,8],lowcut,highcut,fs,order=6)

filtereddf = pd.DataFrame(y)
filtereddf.index = df.index[:1000000]

filtereddf_ref = pd.DataFrame(y_ref)
filtereddf_ref.index = df.index[:1000000]


signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode
signal_filtered_ref = filtereddf_ref.iloc[:,0]


###########################
## Signal de 20s

xminspike = int(np.round(12548*(fs/1000)))
xmaxspike = int(np.round(13000*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

In [16]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - burst_data : part of signal where burst (after bandpass)


part_of_signal = signal_filtered

signal = part_of_signal


In [17]:
signal

%t           
0.00        0.987869
0.02       -1.333552
0.04       -3.391723
0.06       -4.977680
0.08       -5.979127
              ...   
19999.90    1.859480
19999.92    0.955399
19999.94    0.361998
19999.96    0.054497
19999.98   -0.082345
Name: 0, Length: 1000000, dtype: float64

In [18]:
### PLOTTING SIGNAL:

plt.figure()
plt.plot(df.index[:1000000], signal, 'g-', linewidth=2, label='data filtered')
plt.xlabel('Time [ms]')
plt.ylabel('Amplitude [µV]')
plt.title('Filtered Signal')
plt.legend()
plt.grid()
plt.show()

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [35]:
noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 20,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)

In [20]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

In [50]:
spike_info = find_spikes(signal, noise_levels, fs, 
                          window_size = 0.001, 
                          noise_window_size = 0.01,
                          threshold_factor = 3.0,
                          maxseparation = 0.002)

spike_info

,indice_max,indice_min,indice_depass_positif,indice_depass_negatif,indice_1er_depass,indice_zero_central,i_max-i_min,Delta_amplitudes
0,47964,48001,47962,47999,47962,47969,-37,20.426868
1,75832,75817,75831,75815,75815,75826,15,18.969985
2,119331,119346,119330,119344,119330,119338,-15,19.954489
3,142531,142548,142530,142546,142530,142536,-17,19.036584
4,142732,142709,142731,142708,142708,142718,23,18.430977
5,173650,173678,173650,173677,173650,173656,-28,17.918428
6,176880,176846,176879,176844,176844,176870,34,19.050473
7,205444,205421,205443,205421,205421,205438,23,20.153656
8,249898,249934,249898,249933,249898,249916,-36,17.591090
9,316179,316190,316177,316189,316177,316184,-11,19.675061


In [51]:
spike_data = record_spikes(signal, fs, spike_info,
                  'indice_1er_depass',
                  t_before = 0.0015,
                  t_after = 0.0015)

(38, 151)


In [52]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                  'indice_1er_depass',
                  t_before = 0.0015,
                  t_after = 0.0015)

spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                  'indice_zero_central',
                  t_before = 0.002,
                  t_after = 0.002)

In [53]:
plt.plot(df.index[:1000000], signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

No handles with labels found to put in legend.
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [54]:
print_spikes(spike_data,
             t_before_alignement = 0.0015,
             first_spike = 1,
             last_spike = 40,
             fs = 25000)